In [67]:
# Imports, as always...
from os import makedirs, path
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from qiskit_algorithms.utils import algorithm_globals
from typing import Union
import warnings
from datetime import date

# Circuitry.
from qiskit import qpy
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister, ParameterVector
from qiskit.circuit.library import UnitaryGate
from qiskit.quantum_info import random_unitary, random_statevector
from qiskit import transpile

# AWS.
import boto3
from braket.circuits import Circuit 
from braket.devices import LocalSimulator
from braket.tracking import tracker
from qiskit_braket_provider import BraketLocalBackend
from qiskit_ibm_runtime import SamplerV2 as Sampler

# Plotting.
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns

# Scripts.
from scipts.circuit import generate_brickwork_circuit, generate_parameterised_input_layer

# Styling.
sns.set_context('paper')
sns.set_theme(style='darkgrid', palette='Dark2')

# RNG.
def reset_seed(seed):
  if seed is None:
    return

  np.random.seed(seed)
  algorithm_globals.random_seed = seed

# Data save path.
data_save_path = './data/aws/local'
makedirs(data_save_path, exist_ok=True)

# Ignore warnings.
warnings.filterwarnings('ignore')

In [3]:
# Specify a local simulator.
device = LocalSimulator() 

# Bell circuit to check that all is working.
with tracker.Tracker() as money_machine:
    bell = Circuit().h(0).cnot(0, 1) 
    result = device.run(bell, shots=100).result()
    
print(result.measurement_counts)
print(f'Task statistics: {money_machine.quantum_tasks_statistics()}')
print(f'Estimated cost to run this example: {money_machine.simulator_tasks_cost():.5f} USD')

Counter({'11': 53, '00': 47})
Task statistics: {}
Estimated cost to run this example: 0.00000 USD


## Buckets

When running on a managed service or QPU, results of tasks are stored in an *S3 bucket* in your account. Buckets are denoted with the prefix `amazon-braket-` followed by whatever identifying characters you'd like.

> A **bucket** is a container for *objects*, which are files and any associated metadata.

To run one of these services, provide the location of the bucket as a positional arguement in the `.run()` call; e.g.

```
my_bucket = 'example-bucket'        # Bucket name
my_prefix = 'simulation-output'     # Folder name (within the bucket)
folder = (my_bucket, my_prefix)

device = ...
task = device.run(..., folder, ...)
```

I believe you would still have access to the result (i.e. via `task.result()`), but the full information is found in the *Quantum Tasks* section in the console. 

## Generating Data via a *Local* Braket Simulator

In [4]:
# Instantiate a local backend.
local_backend = BraketLocalBackend()

In [5]:
# Brickwork circuit.
brickwork_circuit = generate_brickwork_circuit(n=5, d=4, seed=42)
brickwork_circuit.draw()

┌───────────┐             ┌───────────┐             
q_0: ┤0          ├─────────────┤0          ├─────────────
     │  $U_{00}$ │┌───────────┐│  $U_{20}$ │┌───────────┐
q_1: ┤1          ├┤0          ├┤1          ├┤0          ├
     ├───────────┤│  $U_{11}$ │├───────────┤│  $U_{31}$ │
q_2: ┤0          ├┤1          ├┤0          ├┤1          ├
     │  $U_{01}$ │├───────────┤│  $U_{21}$ │├───────────┤
q_3: ┤1          ├┤0          ├┤1          ├┤0          ├
     └───────────┘│  $U_{12}$ │└───────────┘│  $U_{32}$ │
q_4: ─────────────┤1          ├─────────────┤1          ├
                  └───────────┘             └───────────┘

In [52]:
# Full experiment circuit (depth-varied).
def build_experiment_circuit(brickwork_circuit : QuantumCircuit, parameterised_input : bool = False):
    # Circuit to build up.
    n = brickwork_circuit.num_qubits
    experiment_circuit = QuantumCircuit(n)
    
    # Add input preparation layer if necessary.
    if parameterised_input:
        experiment_circuit = experiment_circuit.compose(generate_parameterised_input_layer(n))
        experiment_circuit.barrier()
    
    # Add the meat.    
    experiment_circuit = experiment_circuit.compose(brickwork_circuit)
    
    # Measure.
    experiment_circuit.measure_all()
    
    return experiment_circuit

experiment_circuit = build_experiment_circuit(brickwork_circuit, parameterised_input=True   )
experiment_circuit.draw()

┌─────────────┐ ░ ┌───────────┐             ┌───────────┐             »
   q_0: ┤ Rx(π*in[0]) ├─░─┤0          ├─────────────┤0          ├─────────────»
        ├─────────────┤ ░ │  $U_{00}$ │┌───────────┐│  $U_{20}$ │┌───────────┐»
   q_1: ┤ Rx(π*in[1]) ├─░─┤1          ├┤0          ├┤1          ├┤0          ├»
        ├─────────────┤ ░ ├───────────┤│  $U_{11}$ │├───────────┤│  $U_{31}$ │»
   q_2: ┤ Rx(π*in[2]) ├─░─┤0          ├┤1          ├┤0          ├┤1          ├»
        ├─────────────┤ ░ │  $U_{01}$ │├───────────┤│  $U_{21}$ │├───────────┤»
   q_3: ┤ Rx(π*in[3]) ├─░─┤1          ├┤0          ├┤1          ├┤0          ├»
        ├─────────────┤ ░ └───────────┘│  $U_{12}$ │└───────────┘│  $U_{32}$ │»
   q_4: ┤ Rx(π*in[4]) ├─░──────────────┤1          ├─────────────┤1          ├»
        └─────────────┘ ░              └───────────┘             └───────────┘»
meas: 5/══════════════════════════════════════════════════════════════════════»
                                                                              »
«         ░ ┌─┐            
«   q_0: ─░─┤M├────────────
«         ░ └╥┘┌─┐         
«   q_1: ─░──╫─┤M├─────────
«         ░  ║ └╥┘┌─┐      
«   q_2: ─░──╫──╫─┤M├──────
«         ░  ║  ║ └╥┘┌─┐   
«   q_3: ─░──╫──╫──╫─┤M├───
«         ░  ║  ║  ║ └╥┘┌─┐
«   q_4: ─░──╫──╫──╫──╫─┤M├
«         ░  ║  ║  ║  ║ └╥┘
«meas: 5/════╩══╩══╩══╩══╩═
«            0  1  2  3  4

In [58]:
# Set up.
sampler = Sampler(mode=local_backend)
x_in = [0, 1, 0, 1, 1]

# Bind parameters.
bounded_circuit = experiment_circuit.assign_parameters({
    f'in[{i}]' : x_in[i] for i in range(experiment_circuit.num_parameters)
})

# Run and show.
job = local_backend.run(bounded_circuit, shots=100)
print(job.result().get_counts())

{'00011': 6, '11111': 10, '00111': 15, '11001': 2, '10010': 4, '10000': 4, '00110': 5, '00001': 6, '11101': 8, '10111': 2, '11000': 6, '01000': 1, '00101': 4, '01011': 2, '10100': 2, '11110': 4, '01001': 4, '10011': 6, '11011': 1, '00000': 2, '01110': 1, '11010': 1, '00100': 1, '00010': 1, '10110': 1, '01101': 1}


In [90]:
def aws_routine(circuit : QuantumCircuit, backend, n_runs: int, parameter_sets : Union[np.array, None], n_shots: int = 1000, write_to_file: bool = False, file_name : str = 'data', seed : Union[int, None] = None, show_progress_bar : bool = True) -> pd.DataFrame:
    # Checking validity of the parameter sets.
    if parameter_sets is not None:
        assert parameter_sets.shape[1] == circuit.num_parameters
    
    # RNG.
    reset_seed(seed)

    # Chained circuit.
    experiment_circuit = build_experiment_circuit(brickwork_circuit, parameterised_input=True)

    # Infer circuit configuration.
    n, d = brickwork_circuit.num_qubits, brickwork_circuit.depth()
  
    # Dataframe to hold results.
    results_df = pd.DataFrame(
        columns=['n', 'd', 'run', 'x_in', 'n_shots', 'device'] + [format(i, f'0{n}b') for i in range(2 ** n)])

    if write_to_file:
        # Create a subdirectory for the backend if none exists.
        makedirs(path.join(data_save_path, backend.backend_name), exist_ok=True)

        # File name.
        file_path = path.join(
            data_save_path, file_name.split('.')[0] + '.csv'
        )

        # Empty file.
        results_df.to_csv(file_path, index=False, header=True)
        
    for run in (tqdm(range(1, n_runs + 1), desc='Runs', leave=False) if show_progress_bar else range(1, n_runs + 1)):
        for x_in in (tqdm(parameter_sets, desc='Parameter Sets', leave=False) if show_progress_bar else parameter_sets):
            # Bind parameters.
            bounded_circuit = experiment_circuit.assign_parameters({
                f'in[{i}]' : x_in[i] for i in range(experiment_circuit.num_parameters)
            })
            
            # Run (via backend).
            job = local_backend.run(bounded_circuit, shots=n_shots)
            
            # Convert into an outcome distribution.
            outcomes = {
              key: value / n_shots
              for key, value in job.result().get_counts().items()
            }

            # Compact into a DataFrame object.
            run_result_df = pd.DataFrame(
                columns=['n', 'd', 'run', 'n_shots', 'device'] + [format(i, f'0{n}b') for i in range(2 ** n)]
            )
            
            # Translate the entries of the outcome distribution into the DataFrame.
            # Any outcome states not measured will be left as NaN.
            for state, prob in outcomes.items():
                run_result_df[state] = [prob]

            # "Meta" data.
            run_result_df['n'] = [n]
            run_result_df['d'] = [d]
            run_result_df['run'] = [run]
            run_result_df['x_in'] = [''.join(x_in.astype(str))]
            run_result_df['n_shots'] = [n_shots]
            run_result_df['device'] = [backend.backend_name]
            
            # Replace NaN with 0.
            run_result_df.fillna(0., inplace=True)

            # Add to the overall results.
            results_df = pd.concat([results_df, run_result_df], ignore_index=True)

            # Write to file.
            if write_to_file:
                run_result_df.to_csv(file_path, mode='a', index=False, header=False)

    return results_df

In [94]:
test_df = aws_routine(
    circuit=experiment_circuit,
    backend=local_backend,
    n_runs=10,
    parameter_sets=np.array([[0, 1, 1, 1, 0], [1, 1, 1, 0, 0], [1, 0, 1, 1, 0]]),
    n_shots=100,
    write_to_file=True,
    file_name='test',
    seed=42,
    show_progress_bar=False
)

In [95]:
test_df

,n,d,run,x_in,n_shots,device,00000,00001,00010,00011,...,10110,10111,11000,11001,11010,11011,11100,11101,11110,11111
0,5,4,1,01110,100,default,0.01,0.06,0.07,0.07,...,0.00,0.05,0.01,0.01,0.03,0.08,0.02,0.01,0.02,0.01
1,5,4,1,11100,100,default,0.09,0.01,0.03,0.02,...,0.05,0.01,0.05,0.02,0.00,0.01,0.02,0.03,0.00,0.00
2,5,4,1,10110,100,default,0.00,0.12,0.01,0.04,...,0.00,0.00,0.04,0.11,0.00,0.03,0.02,0.02,0.04,0.01
3,5,4,2,01110,100,default,0.02,0.04,0.04,0.10,...,0.03,0.04,0.01,0.00,0.01,0.07,0.03,0.02,0.03,0.01
4,5,4,2,11100,100,default,0.16,0.05,0.05,0.02,...,0.01,0.02,0.04,0.04,0.01,0.00,0.01,0.01,0.00,0.00
5,5,4,2,10110,100,default,0.00,0.17,0.00,0.04,...,0.00,0.00,0.05,0.09,0.01,0.04,0.03,0.04,0.00,0.04
6,5,4,3,01110,100,default,0.00,0.07,0.05,0.04,...,0.01,0.02,0.00,0.03,0.04,0.06,0.08,0.02,0.05,0.01
7,5,4,3,11100,100,default,0.05,0.04,0.06,0.04,...,0.02,0.01,0.01,0.05,0.00,0.01,0.00,0.01,0.01,0.00
8,5,4,3,10110,100,default,0.00,0.19,0.00,0.03,...,0.00,0.00,0.07,0.09,0.00,0.02,0.05,0.01,0.01,0.02
9,5,4,4,01110,100,default,0.02,0.04,0.03,0.05,...,0.02,0.02,0.02,0.01,0.02,0.10,0.11,0.02,0.03,0.01
